<a href="https://colab.research.google.com/github/Randrita/deep_learning_projects/blob/main/Bert_Data_fetch_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.5MB/s eta 0:13:09tcmalloc: large alloc 1147494400 bytes == 0x5562cee32000 @  0x7f057eef4615 0x556296976cdc 0x556296a5652a 0x556296979afd 0x556296a6afed 0x5562969ed988 0x5562969e84ae 0x55629697b3ea 0x5562969ed7f0 0x5562969e84ae 0x55629697b3ea 0x5562969ea32a 0x556296a6be36 0x5562969e9853 0x556296a6be36 0x5562969e9853 0x556296a6be36 0x5562969e9853 0x556296a6be36 0x556296aee3e1 0x556296a4e6a9 0x5562969b9cc4 0x55629697a559 0x5562969ee4f8 0x55629697b30a 0x5562969e93b5 0x5562969e87ad 0x55629697b3ea 0x5562969e93b5 0x55629697b30a 0x5562969e93b5
     |█████████████████               | 1055.7MB 1.4MB/s eta 0:11:14tcmalloc: large alloc 1434370048 bytes == 0x556313488000 @  0x7f057eef4615 0x556296976cdc 0x556296a5652a 0x556296979afd 0x556296a6afed 0x5562969ed988 0x5562969e84ae 0x55629697b3ea 0x5562969ed7f0 0x5562969e84ae 0x55629697b3ea 0x5562969ea32a 0x556296a6be36 0x5562969e9853 0

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy


     |████████████████████████████████| 2.3MB 32.3MB/s 
     |████████████████████████████████| 901kB 24.5MB/s 
     |████████████████████████████████| 3.3MB 23.0MB/s 


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
#model_initialisation
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [5]:
#encode_sentiment_calculation

tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')


In [6]:
result = model(tokens)


In [7]:
result.logits


tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [8]:
int(torch.argmax(result.logits))+1


4

In [18]:
#collect_reviews
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [27]:
#Load Reviews into DataFrame and Score
import numpy as np
import pandas as pd


In [26]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [28]:
df['review'].iloc[0]


'Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.'

In [29]:
df.tail()

,review
6,We came for brunch twice in our week-long visi...
7,This is my fave brunch café in and around Sydn...
8,Delicious. The waitress was hot. The burger wa...
9,Rainy day in Sydney. Borrowed an umbrella and ...
10,This cafe is one of the most popular cafes whe...


In [30]:
#calculate_sentiment
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [31]:
sentiment_score(df['review'].iloc[1])


5

In [32]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,Still one of the favorite coffee shop in Sydne...,5
1,Good coffee and toasts. Straight up and down -...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,Ron & Jo are on the go down under and Wow! We...,5
4,Great coffee and vibe. That's all you need. C...,5
5,Great coffee and vibe. That's all you need. C...,4
6,We came for brunch twice in our week-long visi...,4
7,This is my fave brunch café in and around Sydn...,5
8,Delicious. The waitress was hot. The burger wa...,4
9,Rainy day in Sydney. Borrowed an umbrella and ...,5
